# SAN Assignment - regression

Author : Your Name \
Email  : you@fel.cvut.cz

## Submission
Fill in your name above for clarity.
To solve this homework, simply write your answers into this document and fill in the marked pieces of code.
Submit your solution consisting of both this modified notebook file and exported PDF document as an archive to the courseware BRUTE upload system for the SAN course.
The deadline is specified there. 

## Initialization

Load the required libraries `numpy`, `statsmodels` and `sklearn`, make sure you have those installed. We also fix the random seed for reproducibility convenience.

In [1]:
import numpy as np
import statsmodels.api as sma
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

seed = 0

np.random.seed(seed)
rng = np.random.default_rng(seed=seed)
rs = np.random.RandomState(seed=seed)

Here, we define constants of the assignment.
You may play with the values and observe what happens,
but in your solution you should use the given values unchanged.

In [2]:
n_samples = 256     # Total number of samples (training and testing together)
n_dimensions = 100  # Number of n.dimensions, a.k.a. attributes or features

### Model evaluation procedure

The function `learn_and_test` takes a matrix of independent variables values `X` and a corresponding vector
of dependent variable (a.k.a. response) `y` then trains and evaluates a model specified by the `modelType` parameter.

Unlike in R, in Python you have much higher control over your models and have to be explicit. That is why this function is prepared for you specially for this assignment and is prepared to provide you interface to three simple models---`"ols"` for ordinary least squares, `"lasso"` for LASSO and `"ridge"` for Ridge regression---including the cross-validation to select the best regularization parameter alpha for the regularized models. 
The models all come from the statsmodels package and you can refer to their documentation at https://www.statsmodels.org, the models are specific instants of the OLS model.
For purpose of this assignment, consider only the RMSE (root-mean-square error) criterion. 

In [3]:
def learn_and_test(X, y, model_type, alphas=None):
    assert model_type in ["ols", "lasso", "ridge"]

    if model_type == "ols":
        res = sma.OLS(endog=y, exog=X).fit()
        print(res.summary())
    else:
        assert alphas is not None

        n_splits = 10
        print()
        print(" " * 10 + "GLS regression results")
        print("=" * 50)
        print(f"Model type:  {model_type}")
        print(f"Cross-validation results ({n_splits} splits)")
        print(r"     alpha           MSE         R2          MAE")

        skf = KFold(n_splits=n_splits, shuffle=True, random_state=rs)
        mses_over_alphas = []
        for a in alphas:
            total_mse = 0
            total_r2 = 0
            total_mae = 0
            for train_idxs, test_idxs in skf.split(X, y):
                model = sma.OLS(endog=y[train_idxs], exog=X[train_idxs])
                fit = model.fit_regularized(alpha=a, L1_wt=1 if model_type == "lasso" else 0)
                cmp = y[test_idxs], fit.predict(X[test_idxs])
                total_mse += mean_squared_error(*cmp)
                total_mae += mean_absolute_error(*cmp)
                total_r2 += r2_score(*cmp)
            mean_mse = total_mse / skf.get_n_splits()
            mean_r2 = total_r2 / skf.get_n_splits()
            mean_mae = total_mae / skf.get_n_splits()
            print(f"  {a: >10,.4e}  |  {mean_mse: >10,.3f}   {mean_r2:>6,.8f}   {mean_mae:>10,.3f}")

            mses_over_alphas.append(mean_mse)
        best = np.argmin(mses_over_alphas)
        res = sma.GLS(endog=y, exog=X).fit_regularized(alpha=alphas[best], L1_wt=1 if model_type == "lasso" else 0)

    return res


We will also precompute an array of candidate alpha values for LASSO and Ridge.

In [4]:
alphas = 10 ** np.linspace(10, -3, num=10)

### Initial data generation

Here, we generate some data.

In [5]:
# Generates independent variables by uniform i.i.d. sampling
X = rng.uniform(low=-10, high=10, size=(n_samples, n_dimensions))

# Randomly generates the actual underlying coefficients of the linear dependency
coefs = rng.uniform(low=1, high=4, size=(n_dimensions,)) 
intercept = 0 # For simplicity

# Synthesizes dependent variable (observed values) by the given linear dependency plus noise
noise = rng.normal(loc = 0, scale = 8, size=(n_samples)) # Gaussian noise to be added to the response
y = (X @ coefs) + intercept + noise

## Testing the models

Now let us run the following tests. Note, that in Python implementation of the $R^2$ score, the computed value can be negative, which means that the trained model is worse than uninformed one. Reflect on why that can happen and why do we see this for the regularized models.

In [6]:
learn_and_test(X, y, "ols")
learn_and_test(X, y, "lasso", alphas = alphas)
learn_and_test(X, y, "ridge", alphas = alphas)

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                              1065.
Date:                Mon, 10 Oct 2022   Prob (F-statistic):                   3.06e-186
Time:                        15:19:01   Log-Likelihood:                         -818.96
No. Observations:                 256   AIC:                                      1838.
Df Residuals:                     156   BIC:                                      2192.
Df Model:                         100                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

### Task 1:
**Answer the following questions:**

  * What change in the learned model would you anticipate if we changed the `mean` parameter value to a different constant in the noise generation? You may answer this either by talking about the coefficients or by giving a geometrical interpretation. **Answer:**  
  * In our example we generated samples (i.e. the independent variables `X`) from uniform distribution. The least squares  method, on the other hand, has something called the "normality assumption". Have we violated that assumption? Justify. **Answer:** 
  * Which method gave the best results? Is it the most common one to do so if you re-run the test several times? Why do you think it performs better the best? **Answer:** 
  * Check the selected values for the lambda parameter for ridge and LASSO. Are they low or high? How does it relate to the above answer? **Answer:** 

## Least squares assumptions

The data generation model assumed by the ordinary least squares method can be mathematically written as follows:
$$Y = \mathbf{X}^T \boldsymbol{\beta} + \beta_0 + G, \; G \sim \mathcal{N}(0, \sigma^2)$$
This formula implicitly expresses some of the assumptions about the data,
required for the method to work reliably.  

  * The observed value $Y$ is influenced by some Gaussian noise $G$.  
  * There truly exists an underlying linear dependency. 
  * The noise is homoscedastic ($\sigma^2$ is a constant).
  
### Task 2:
First of all, make sure you understand how elements of this formula correspond to the code in the "data generation" section.

Your task is to violate each of these assumptions (one at a time) and **briefly** comment the changes in the learned model by statistically comparing it to model using the above data. (Coefficient summary below.) The catch here is that you are allowed to only modify the noise generation procedure to achieve that. Attempt to find a way of violating the assumptions to achieve a clear difference, but any solution that is technically correct will be awarded full points. 

It is sufficient to look at the `summary` of the OLS model.

In [ ]:
learn_and_test(X, y, "ols")

#### Violate noise normality

In [ ]:
noise = rng.normal(loc=0, scale=8, size=n_samples) # REPLACE THIS WITH YOUR CODE

# KEEP THE CODE BELOW
y = X @ coefs + intercept + noise
learn_and_test(X, y, "ols")

**Your comment:**

#### Violate linearity

In [ ]:
noise = rng.normal(loc=0, scale=8, size=n_samples) # REPLACE THIS WITH YOUR CODE

# KEEP THE CODE BELOW
y = X @ coefs + intercept + noise
learn_and_test(X, y, "ols")

**Your comment:** 

#### Violate homoscedasticity

In [ ]:
noise = rng.normal(loc=0, scale=8, size=n_samples) # REPLACE THIS WITH YOUR CODE

# KEEP THE CODE BELOW
y = X @ coefs + intercept + noise
learn_and_test(X, y, "ols")

**Your comment:** 

## Understanding the advantages of shrinkage methods 

In this part, we will be modifying the dependent variables `X` from task 1 by a linear transformation, represented by a square matrix of `n.dimensions` sides. 
For demonstration, consider that the identity function represented by the identity matrix:

In [ ]:
M0 = np.eye(n_dimensions)                                   # Makes an identity matrix
noise = rng.normal(loc=0, scale=8, size=n_samples)          # Sample noise
X0 = X @ M0                                                 # You can check that X0 == X
Y0 = X0 @ coefs + noise                                     # We can reuse the noise as it's independent of X

### Task 3

In this task, you will show your understanding of the advantages of Ridge by synthesizing data on which they perform the best. You are supposed do this by linearly transforming the dataset `X` as in the example above. In other words, you should construct a matrix which if used in place of `M0` in the above example would make Ridge perform better than the other two methods. You should not resort to degenerate cases where you would get a warning about using a rank-deficient matrix. Justify your method.

**Scoring note:** The difference in the RMSE criterion doesn't need to be large or does not need to be present if you are certain it's just an statistical artifact (which you could verify by re-running the tests multiple times or using the LOOCV in `learnAndTest` function). Your design and justification is what matters for the assignment evaluation and the measurements are here only to guide you. We expect this task to be challenging to students, but once you get the right idea, it is possible to implement it with very small amount of code. 

#### Ridge

In [ ]:
# REPLACE
M1 = np.eye(n_dimensions)

# KEEP THE CODE BELOW
noise = rng.normal(loc=0, scale=8, size=n_samples)
X1 = X @ M1
Y1 = X1 @ coefs + noise

Running these test should now make Ridge perform the best. 

In [ ]:
learn_and_test(X1, Y1, "ols")
learn_and_test(X1, Y1, "ridge", alphas=alphas)
learn_and_test(X1, Y1, "lasso", alphas=alphas)

**Justification:** 

#### LASSO (OPTIONAL CHALLENGE)
This part of the homework is purely optional, but we are eager to see students capable of solving this. 
Here you should do the same thing as above, but to make perform LASSO the best of the three methods.
Although similar in nature, we consider this even more challenging than the above
since being unable to modify the underlying coefficients, 
this may require some deeper considerations to justify the transformation method.
It can still be implemented with a few short lines of code, though.